In [1]:
# Create gender concepts using wikipedia

In [2]:
import pandas as pd
import numpy as np
import pyarrow as pa
from datasets import load_from_disk, Dataset
import os

In [3]:
path_dir = '/work3/s174498/concept_random_dataset/wikipedia_20220301/train'
wikipedia = load_from_disk(path_dir)

titles = wikipedia['title']

### Create concepts (Level II and Level III)

In [18]:
def main_articles(list_of_interest, folder_name):

    count_datasets = 0
    concept_list = []
    concept_list_saved = []
    concept_list_notsaved = []

    for i in range(len(list_of_interest)): 
        if list_of_interest[i] in titles:
            concept_list.append(list_of_interest[i])
            idx = titles.index(list_of_interest[i])

            # Clean up data (\n references in wiki article)
            text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
            text_clean2 = text_clean.replace('\n', ' ')
            phrase_to_list = text_clean2.split(". ")
            
            # make sure each phase consists of text
            word_in_sentence = []
            phrase_to_list_reduced = []
            for j in range(len(phrase_to_list)):
                word_in_sentence.append(len(phrase_to_list[j].split()))

                if(word_in_sentence[j] > 1):
                    phrase_to_list_reduced.append(phrase_to_list[j])

            if len(phrase_to_list_reduced) > 150:
                print(list_of_interest[i])
                count_datasets = count_datasets + 1
                concept_list_saved.append(list_of_interest[i])

                df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
                'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list_reduced})
                data_concept = Dataset(pa.Table.from_pandas(df))

                # save data
                data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/' + folder_name + list_of_interest[i])
            else: 
                concept_list_notsaved.append(list_of_interest[i])
        else: 
            print(list_of_interest[i] + ' does not exist')

def sub_articles(list_of_interest, folder_name):
    count_datasets = 0
    concept_list = []
    concept_list_saved = []
    concatenate_phrases = []

    for i in range(len(list_of_interest)): 
        if list_of_interest[i] in titles:
            concept_list.append(list_of_interest[i])
            idx = titles.index(list_of_interest[i])

            # Clean up data (\n references in wiki article)
            text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
            text_clean2 = text_clean.replace('\n', ' ')
            phrase_to_list = text_clean2.split(". ")

            # make sure each phase consists of text
            word_in_sentence = []
            phrase_to_list_reduced = []
            for j in range(len(phrase_to_list)):
                word_in_sentence.append(len(phrase_to_list[j].split()))

                if(word_in_sentence[j] > 1):
                    phrase_to_list_reduced.append(phrase_to_list[j])

            # Save concepts to "Level II"
            concatenate_phrases = concatenate_phrases + phrase_to_list_reduced

            # Save concepts to "Level III"
            if len(phrase_to_list_reduced) > 110:
                print(list_of_interest[i])
                concept_list_saved.append(list_of_interest[i])

                df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
                'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list_reduced})
                data_concept = Dataset(pa.Table.from_pandas(df))

                # save data
                path = os.path.join('/work3/s174498/concept_random_dataset/wikipedia_20220301/', folder_name)
                data_concept.save_to_disk(path + '/' + list_of_interest[i])

    if len(concatenate_phrases) > 110:
        df_levelII = pd.DataFrame({'title': folder_name, 'text_list': concatenate_phrases})
        data_concept_levelII = Dataset(pa.Table.from_pandas(df_levelII))

        # save data
        data_concept_levelII.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/' + folder_name)

### Create function to create concepts (level I)

In [19]:
def create_concept_levelI(path_of_concepts, folder_name):

    list_of_folders = [os.path.join(path_of_concepts, o) for o in os.listdir(path_of_concepts) 
                        if os.path.isdir(os.path.join(path_of_concepts,o))]

    concat_text = []
    for i in range(len(list_of_folders)):
        data = load_from_disk(list_of_folders[i])
        data_text = data['text_list']

        concat_text = concat_text + data_text
        
        df_levelI = pd.DataFrame({'title': folder_name, 'text_list': concat_text})
        data_concept_levelI = Dataset(pa.Table.from_pandas(df_levelI))

        # save data
        data_concept_levelI.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/' + folder_name)

# Create datasets for gender concepts

In [20]:
man_male = ['Man', 'Male']

In [21]:
folder_name = 'gender_concepts_new/man_male'
sub_articles(man_male, folder_name)

In [22]:
woman_female = ['Woman', 'Female']
folder_name = 'gender_concepts_new/woman_female'
sub_articles(woman_female, folder_name)

Woman


In [23]:
intersex = ['Intersex']
folder_name = 'gender_concepts_new/intersex'
sub_articles(intersex, folder_name)

Intersex


# Create datasets for sports concepts

Acrobatic sports: No 'Main articles'. Thus, all sub-articles are concatenated into one Level II concept, while these sub-articles (if longer than 150 sentences) are saved for Level III concepts.

In [6]:
acrobatic_sports = ['Ballet',
'Breakdancing',
'Competitive dancing',
'Cheerleading',
'Dancesport',
'Dragon dance',
'Figure skating',
'Freerunning',
'Gymnastics',
'High kick',
'Parkour',
'Pole sports',
'Stunt',
'Trampolining',
'Winter guard']

In [7]:
folder_name = 'sports_concepts/Acrobatic sports'
sub_articles(acrobatic_sports, folder_name)

Ballet
Breakdancing
Cheerleading
Figure skating
Gymnastics
Parkour


Next, try all conceps having a 'main articles'

In [8]:
list_of_sport = ['Air sports',
'Archery', 'Boardsport']

In [9]:
folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Archery


Air sports: Not enought sentences in 'Main articles'. Thus, all sub-articles are concatenated into one Level II concept, while these sub-articles (if longer than 150 sentences) are saved for Level III concepts.

In [10]:
Air_sports = ['Aerobatics',
'Air racing',
'Gliding',
'Hang gliding',
'Human powered aircraft',
'Model aircraft',
'Parachuting',
'Paragliding',
'Paramotoring',
'Ultralight aviation']

In [11]:
folder_name = 'sports_concepts/Air sports'
sub_articles(Air_sports, folder_name)

Gliding
Hang gliding
Model aircraft
Parachuting
Paragliding


In [12]:
Boardsport = ['Skateboarding',
'Scootering',
'Skysurfing',
'Streetluge',
'Snowboarding',
'Surfing',
'Wakeboarding',
'Paddleboarding']

In [13]:
folder_name = 'sports_concepts/Boardsport'
sub_articles(Boardsport, folder_name)

Skateboarding
Snowboarding
Surfing


In [14]:
# no main article
Catching_games = ['Dodgeball',
'Frisbee',
'Gaga',
'Keep away',
'Kin-Ball',
'Newcomb ball',
'Quidditch',
'Yukigassen']

folder_name = 'sports_concepts/Catching games'
sub_articles(Catching_games, folder_name)

In [15]:
list_of_sport = ['Climbing', 'Cycle sport', 'Combat sport']
folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

In [16]:
Climbing = ['Abseiling', 'Aid climbing',
'Ice climbing',
'Mixed climbing',
'Mountaineering',
'Rock climbing',
'Bouldering',
'Deep-water soloing',
'Sport climbing',
'Traditional climbing',
'Speed climbing',
'Sawanobori']

folder_name = 'sports_concepts/Climbing'
sub_articles(Climbing, folder_name)

Mountaineering
Rock climbing


In [17]:
Cycling = ['Bicycle', 'Skibobbing', 'Unicycle']

folder_name = 'sports_concepts/Cycling'
sub_articles(Cycling, folder_name)

Bicycle


In [18]:
Combat_sports = ['Aiki-jūjutsu', 'Jujutsu',
'Sambo (martial art)',
'Sumo'
'Wrestling', 
'Bajiquan'
'Bokator'
'Boxing'
'Capoeira'
'Chess boxing'
'Choi Kwang-Do'
'Fujian White Crane'
'Karate'
'Kenpō'
'Kickboxing'
'Lethwei'
'Muay Thai'
'Pradal serey'
'Savate'
'Shaolin Kung Fu'
'Shin-kicking'
'Sikaran'
'Silat'
'Subak'
'Taekkyeon'
'Taekwondo'
'Taidō'
'Tang Soo Do'
'Vovinam'
'Wing Chun'
'Bare knuckle boxing'
]

folder_name = 'sports_concepts/Combat sports'
sub_articles(Combat_sports, folder_name)

Jujutsu


In [19]:
Dog_sport = ['Herding',
'Sheepdog trial',
'Treibball',
'Heelwork to music',
'Musical canine freestyle',
'Obedience trial',
'Rally obedience',
'Schutzhund',
'Bikejoring',
'Canicross',
'Carting',
'Dog scootering',
'Mushing',
'Skijoring',
'Weight pulling',
'Dachshund racing',
'Greyhound and Whippet racing',
'Jack Russell Terrier',
'Sighthound',
'Sled dog racing',
'Terrier',
'Coon hunting field trial',
'Earthdog trial',
'Field trial',
'Hare coursing',
'Hound trailing',
'Lure coursing',
'Nosework',
'Tracking trial',
'Trail hunting',
'Agility',
'Disc dog',
'Flyball',
'Dog Puller']

folder_name = 'sports_concepts/Dog sports'
sub_articles(Dog_sport, folder_name)

Obedience trial
Jack Russell Terrier


In [20]:
Equestrian_sports = ['Buzkashi',
'Barrel racing',
'Campdrafting',
'Cirit',
'Charreada',
'Chilean rodeo',
'Chuckwagon racing',
'Cross country',
'Cutting',
'Dressage',
'Endurance riding',
'English pleasure',
'Equitation',
'Eventing',
'Equestrian vaulting',
'Gymkhana',
'Harness racing',
'Hobby horse polo',
'Horse racing',
'Horse polo',
'Horseball',
'Jineteada gaucha',
'Jousting',
'Mounted games',
'Mounted archery',
'Pato',
'Pleasure driving',
'Reining',
'Rodeo',
'Show hunter (British)',
'Show jumping',
'Steeplechase',
'Sur-papakh',
'Team penning',
'Tent pegging',
'Western pleasure']

folder_name = 'sports_concepts/Equestrian sports'
sub_articles(Equestrian_sports, folder_name)

Charreada
Dressage
Eventing
Harness racing
Horse racing
Mounted archery
Rodeo
Show jumping


In [21]:
list_of_sport = ['Fishing', 'Flying disc sports', 'Gymnastics']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Fishing
Gymnastics


In [22]:
Flying_disc_sports = ['Disc dog',
'Disc golf',
'Disc golf (urban)',
'Dodge disc',
'Double disc court',
'Freestyle',
'Frisbee',
'Goaltimate',
'KanJam',
'Guts',
'Hot box',
'Ultimate']

folder_name = 'sports_concepts/Flying disc sports'
sub_articles(Flying_disc_sports, folder_name)

Disc golf


In [23]:
list_of_sport = ['Competitive yoga', 'Hunting']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Hunting


In [24]:
Ice_sports = ['Bandy',
'Barrel jumping',
'Broomball',
'Curling',
'Figure skating',
'Ice cross downhill',
'Ice football',
'Ice hockey',
'Ice yachting',
'Moscow broomball',
'Ringette',
'Rinkball',
'Speed skating',
'Ice stock sport']

folder_name = 'sports_concepts/Ice sports'
sub_articles(Ice_sports, folder_name)

Bandy
Curling
Figure skating
Ice hockey
Ringette


In [25]:
Kite_sports = ['Hang gliding',
'Kite buggy',
'Kite fighting',
'Kite landboarding',
'Kitesurfing',
'Parasailing',
'Snow kiting',
'Sport kite']

folder_name = 'sports_concepts/Kite sports'
sub_articles(Kite_sports, folder_name)

Hang gliding


In [26]:
Mixed_discipline = ['Adventure racing',
'Aquathlon',
'Biathlon',
'Duathlon',
'Decathlon',
'Heptathlon',
'Icosathlon',
'Modern pentathlon',
'Pentathlon',
'Tetrathlon',
'Triathlon'
]

folder_name = 'sports_concepts/Mixed discipline'
sub_articles(Mixed_discipline, folder_name)

Triathlon


In [27]:
Orienteering_family = ['Amateur radio direction finding', 
'Geocaching',
'Geohashing',
'Letterboxing',
'Orienteering',
'Rogaining',
'Waymarking']

folder_name = 'sports_concepts/Orienteering family'
sub_articles(Orienteering_family, folder_name)

Amateur radio direction finding
Geocaching
Orienteering


In [28]:
Parkour_Freerunning = ['Freerunning',
'Tricking',
'Sasuke',
'Obstacle course racing', 
'Parkour',
'Trampolining',
'World Chase Tag'] 

folder_name = 'sports_concepts/Parkour Freerunning'
sub_articles(Parkour_Freerunning, folder_name)

Parkour


In [29]:
list_of_sport = ['Rodeo', 'Running', 'Sailing']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Rodeo
Running
Sailing


In [30]:
Snow_sports = ['Broombal', 'Skiing', 'Sledding'] 

folder_name = 'sports_concepts/Snow sports'
sub_articles(Snow_sports, folder_name)

In [31]:
list_of_sport = ['Shooting sports', 'Tag (game)']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Shooting sports


In [32]:
Stacking = ['House of cards',
'Sport stacking',
'Dice stacking',
'Sport stacking']

folder_name = 'sports_concepts/Stacking'
sub_articles(Stacking, folder_name)

In [33]:
Street_sports = ['Free running',
'Freestyle footbag',
'Freestyle football',
'Jumping stilts',
'Parkour',
'Freestyle scootering',
'Street workout']

folder_name = 'sports_concepts/Street sports'
sub_articles(Street_sports, folder_name)

Parkour


In [34]:
Strength_sports = ['Strongman']

folder_name = 'sports_concepts/Strength sports'
sub_articles(Strength_sports, folder_name)

In [35]:
Tag_game = ['Tag (Game)', 
'Atya-patya',
'Bo-taoshi',
'British Bulldog',
'Capture the flag',
'Darebase',
'Hana Ichi Monme',
'Hide and seek',
'Jugger',
'Kabaddi',
'Kho kho',
'Kick the can',
'Langdi',
'Marco Polo',
'Oztag',
'Red rover',
'World Chase Tag']

folder_name = 'sports_concepts/Tag game'
sub_articles(Tag_game, folder_name)

Jugger
Marco Polo


In [36]:
Walking = ['Backpacking (hiking)',
'Bushwhacking',
'Hiking',
'Racewalking',
'Walking']

folder_name = 'sports_concepts/Walking'
sub_articles(Walking, folder_name)

Hiking
Walking


In [37]:
Aquatic_and_paddle_sports = ['Canoeing',
'Outrigger canoeing', 'Kayaking', 'Rafting',
'White water rafting', 'Rowing (sport)',
'Gig racing',
'Coastal and ocean rowing',
'Single scull', 'Dragon boat racing',
'Stand up paddle boarding', 'Water polo',
'Canoe polo',
'Stone skipping',
'Waboba', 'Underwater football',
'Underwater rugby',
'Underwater hockey',
'Underwater ice hockey', 'Swimming (sport)', 
'Bifins (finswimming)', 
'Surface finswimming', 'Recreational diving', 'diving']

folder_name = 'sports_concepts/Aquatic and paddle sports'
sub_articles(Aquatic_and_paddle_sports, folder_name)

Rowing (sport)
Water polo
Underwater hockey
Underwater ice hockey
Swimming (sport)
Recreational diving


In [38]:
Weightlifting = ['Olympic weightlifting', 'Powerlifting']

folder_name = 'sports_concepts/Weightlifting'
sub_articles(Weightlifting, folder_name)

Powerlifting


In [39]:
list_of_sport = ['Motorsport']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

In [40]:
Motorsport = ['Motorsport', 'Auto racing', 'Offshore powerboat racing', 'Motorcycle racing', 'All-terrain vehicle']

folder_name = 'sports_concepts/Motersport'
sub_articles(Motorsport, folder_name)

Auto racing
Motorcycle racing
All-terrain vehicle


In [41]:
Marker_sports = ['Airsoft',
'Archery',
'Paintball',
'Darts']

folder_name = 'sports_concepts/Marker sports'
sub_articles(Marker_sports, folder_name)

Airsoft
Archery
Paintball
Darts


In [42]:
Overlapping_sports = ['Chess boxing',
'Motoball',
'Whirlyball']

folder_name = 'sports_concepts/Overlapping sports'
sub_articles(Overlapping_sports, folder_name)

In [43]:
Other = ['Singing',
'Band',
'Marching band',
'Beatboxing']

folder_name = 'sports_concepts/Other'
sub_articles(Other, folder_name)

Singing
Marching band


In [44]:
list_of_sport = ['Bat-and-ball games']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

In [45]:
Bat_and_ball_games = ['Baseball',
'Bat and trap',
'Bete-ombro',
'Brännboll',
'Corkball',
'Cricket', 
'Danish longball',
'Donkey Baseball',
'Elle',
'English baseball',
'Kickball',
'Lapta',
'The Massachusetts Game',
'Matball',
'Oină',
'Old cat',
'Over-the-line',
'Palant',
'Pesäpallo',
'Plaquita',
'Punchball',
'Rounders',
'Schlagball',
'Scrub baseball',
'Stickball',
'Stoolball',
'Stoop ball',
'Tee-ball',
'Town ball',
'Vigoro',
'Vitilla',
'Welsh baseball',
'Wiffleball',
'Wireball']

folder_name = 'sports_concepts/Bat and ball games'
sub_articles(Bat_and_ball_games, folder_name)

Baseball
Cricket


In [46]:
list_of_sport = ['Goal (sports)']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

In [47]:
Invasion_games = ['Goal (sports)', 'Basketball',
'Cestoball',
'Flickerball',
'Korfball',
'Netball',
'Ringball',
'Slamball', 'Football', 'Handball', 'Hornussen', 'Hockey', 'Cammag',
'Hurling',
'Shinty',
'Composite rules shinty-hurling', 'Lacrosse', 'Polo'
]

folder_name = 'sports_concepts/Invasion games'
sub_articles(Invasion_games, folder_name)

Basketball
Netball
Football
Handball
Hurling
Lacrosse
Polo


In [48]:
Net_and_wall_games = ['Badminton',
'Ball badminton',
'Biribol',
'Bossaball',
'Fistball',
'Footbag net',
'Football tennis',
'Footvolley',
'Jianzi',
'Jokgu',
'Miniten',
'Padel',
'Peteca',
'Pickleball',
'Rocball',
'Sepak takraw',
'Sipa',
'Table squash',
'Table tennis',
'Tennis',
'Teqball',
'Throwball',
'Volleyball',
'Wallyball',
'American handball',
'Australian handball',
'Basque pelota',
'Butts Up',
'Chinese handball',
'Fives',
'Gaelic handball',
'International fronton',
'Jai alai',
'Jorkyball',
'Racquetball',
'Squash',
'Squash tennis',
'Stoop ball',
'Suicide (game)',
'Valencian frontó',
'Wallball',
'Wallyball',
'American handball',
'Australian handball',
'Basque pelota',
'Fives',
'Frisian handball',
'Four square',
'Gaelic handball',
'Jeu de paume',
'Palla',
'Pallone',
'Patball',
'Roundnet',
'Valencian pilota']

folder_name = 'sports_concepts/Net and wall games'
sub_articles(Net_and_wall_games, folder_name)

Badminton
Fistball
Sepak takraw
Table tennis
Tennis
Volleyball
Racquetball


In [49]:
list_of_sport = ['Card game', 'Esports', 'Speedcubing', 'Board game']

folder_name = 'sports_concepts/'
main_articles(list_of_sport, folder_name)

Card game
Esports
Speedcubing
Board game


In [50]:
Other_mind_sports = ['Competitive programming',
'Debating',
'Fantasy sport',
'Quizzing']

folder_name = 'sports_concepts/Other mind sports'
sub_articles(Other_mind_sports, folder_name)

Fantasy sport


In [51]:
Competitive_model_sports = [
'Mini 4WD',
'Model aircraft racing',
'Model yacht racing',
'Neppis']

folder_name = 'sports_concepts/Competitive model sports'
sub_articles(Competitive_model_sports, folder_name)

In [52]:
Remote_control = ['Drone racing',
'Model aerobatics',
'RC racing',
'Robot combat',
'Robot football',
'Slot car racing']

folder_name = 'sports_concepts/Remote control'
sub_articles(Remote_control, folder_name)

Robot combat
Slot car racing


In [53]:
Air_sports = ['Banzai skydiving',
'Bungee jumping',
'Hot air ballooning']

folder_name = 'sports_concepts/Air sports'
sub_articles(Air_sports, folder_name)

In [54]:
Athletics = ['Athletics',
'Steeplechase',
'Cross country',
'Jumping',
'Throwing',
'Racewalking']

folder_name = 'sports_concepts/Athletics'
sub_articles(Athletics, folder_name)

In [55]:
Electronic_sports = ['Combat robot',
'Competitive programming',
'Contesting',
'Fitness game',
'Radio-control vehicles',
'Video gaming']

folder_name = 'sports_concepts/Electronic sports'
sub_articles(Electronic_sports, folder_name)

Contesting


In [56]:
Skating_sports = ['Bandy',
'Figure skating',
'Ice cross downhill',
'Ice dancing',
'Ice hockey',
'Ice skating',
'Inline skating',
'Roller skating',
'Ringette',
'Rinkball',
'Speed skating',
'Tour skating']

folder_name = 'sports_concepts/Skating sports'
sub_articles(Skating_sports, folder_name)

Bandy
Figure skating
Ice hockey
Roller skating
Ringette


In [57]:
Strength_sports = ['Arm wrestling',
'CrossFit',
'Highland games',
'Olympic weightlifting',
'Powerlifting',
'Rock climbing',
'Strongman',
'Thumb wrestling',
'Toe wrestling',
'Tug-o-war',
'Wood chopping',
'Wood splitting',
'Zourkhaneh',
'Calisthenics']

folder_name = 'sports_concepts/Strength sports'
sub_articles(Strength_sports, folder_name)

Powerlifting
Rock climbing


In [58]:
Table_sports = ['Air hockey',
'Backgammon',
'Beer Pong',
'Carrom Nok Hockey',
'Chess',
'Connect Four',
'Cue sports',
'Draughts',
'Dominoes',
'Headis',
'Janggi',
'Mahjong',
'Paper Football',
'Reversi',
'Scrabble',
'Shogi',
'Subbuteo',
'Table football',
'Table hockey',
'Table tennis',
'Table Shuffleboard/Sling Puck',
'Teqball',
'Xiangqi']

folder_name = 'sports_concepts/Table sports'
sub_articles(Table_sports, folder_name)

Backgammon
Chess
Cue sports
Dominoes
Mahjong
Scrabble
Shogi
Table tennis
Xiangqi


In [59]:
Target_sport = ['Archery', 'Bowling', 'Cue sports', 'Golf']

folder_name = 'sports_concepts/Target sport'
sub_articles(Target_sport, folder_name)

Archery
Bowling
Cue sports
Golf


### Create 'Level I' concept: Sport

In [60]:
path_of_sports = '/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/'
folder_name = 'sports_concepts/'

create_concept_levelI(path_of_sports, folder_name)

In [61]:
# load_from_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/')